In [3]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from sklearn.cluster import KMeans
import holidays

# === 0) COLAB FILE UPLOAD ===
from google.colab import files
import io

# Prompt user to upload the two CSVs
print("Please upload the hourly power data and the temperature data files:")
uploaded = files.upload()

# Load uploaded files into DataFrames
power_df = pd.read_csv(
    io.BytesIO(uploaded['combined_power_data_dottenijs_hourly.csv']),
    parse_dates=[0], index_col=0
)
df_temp = pd.read_csv(
    io.BytesIO(uploaded['BEITEM_temp_data_updated_final (2).csv']),
    parse_dates=['timestamp'],
)

Please upload the hourly power data and the temperature data files:


Saving BEITEM_temp_data_updated_final (2).csv to BEITEM_temp_data_updated_final (2) (1).csv
Saving combined_power_data_dottenijs_hourly.csv to combined_power_data_dottenijs_hourly (1).csv


KeyError: 'combined_power_data_dottenijs_hourly.csv'

In [9]:
power_df.rename(columns={'Price': 'Load'}, inplace=True)

# Prepare temperature series
df_temperature = df_temp['temp_grass_pt100_avg']

# === 1) MERGE & INITIAL CLEANUP ===
# Use power_df as primary dataset
power_df.index = pd.to_datetime(power_df.index)
data = power_df.copy()

# === 2) BASIC TIME FEATURES ===
# Raw time components
data['hour']         = data.index.hour
data['Day_of_week']  = data.index.dayofweek
data['Day_of_year']  = data.index.dayofyear
data['month']        = data.index.month
data['year']         = data.index.year
# ISO week number
data['week_of_year'] = data.index.isocalendar().week.astype(int)
# Season mapping: 0=Summer,1=Autumn,2=Winter,3=Spring
data['season'] = data['month'].map(lambda m: 2 if m in [12,1,2]
                                            else 3 if m in [3,4,5]
                                            else 0 if m in [6,7,8]
                                            else 1)
# Weekend flag
data['is_weekend'] = data['Day_of_week'].isin([5,6]).astype(int)

# === 3) CYCLICAL ENCODING ===
# Hour of day
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
# Day of week
data['day_sin']  = np.sin(2 * np.pi * data['Day_of_week'] / 7)
data['day_cos']  = np.cos(2 * np.pi * data['Day_of_week'] / 7)

# === 4) HOLIDAYS ===
# Belgium holiday calendar
data['Holidays'] = data.index.to_series().apply(
    lambda x: int(bool(holidays.CountryHoliday('BE').get(x)))
)

# === 5) ROLLING STD & MEAN & HISTORICAL AVG ===
std_windows = [168, 120, 48, 24, 12, 4]
hist_specs  = [(24, ""), (14, "_direct")]
mean_specs  = [(1,24), (1,4)]
for shift_h, suffix in hist_specs:
    for w in std_windows:
        data[f'roll_{w}_std{suffix}'] = data['Load'].shift(shift_h).rolling(w).std()
        data[f'hist_avg{suffix}_{w}'] = data['Load'].shift(shift_h).rolling(w, min_periods=1).mean()
for shift_h, win in mean_specs:
    data[f'roll_{win}_mean'] = data['Load'].shift(shift_h).rolling(win).mean()

# === 6) LAG FEATURES ===
for lag in [24,48,144,168,312,336]:
    data[f'lag_{lag}'] = data['Load'].shift(lag)

# === 7) TREND & AVG LAST PERIODS ===
def compute_slope(arr):
    return linregress(np.arange(len(arr)), arr).slope
for w in [4,24,48]:
    data[f'trend_last{w}h'] = data['Load'].shift(24).rolling(w).apply(compute_slope, raw=True)
    data[f'avg_last{w}h']   = data['Load'].shift(24).rolling(w).mean()

# === 8) DAY-OF-WEEK ONE-HOT ===
# Remove old dow_* if any
data = data.drop(columns=[c for c in data if c.startswith('dow_')], errors='ignore')
dow_d = pd.get_dummies(data['Day_of_week'], prefix='dow', dtype=int)
data = data.join(dow_d)

# === 9) EXOGENOUS STATE FLAGS ===
offset_min, offset_med = 100, 400
rmx = data['Load'].rolling(2, min_periods=1).max()
rmin = data['Load'].rolling(2, min_periods=1).min()
data['Exogenous_3_off']     = (rmx <= offset_min).astype(int)
data['Exogenous_4_standby'] = ((rmin > offset_min) & (rmx <= offset_med)).astype(int)

# === 10) CLUSTER RELATION HOUR-WEEKDAY ===
# Use training subset for clustering
data_clust = data.loc[data.index < pd.to_datetime('2024-10-07')]
# Compute median Load per (Day_of_week, hour)
gs = data_clust.groupby(['Day_of_week','hour'])['Load'].median().reset_index()
kmeans = KMeans(n_clusters=3, random_state=42)
gs['cluster'] = kmeans.fit_predict(gs[['Load']])
label_map = {0:2, 2:0, 1:1}
gs['cluster'] = gs['cluster'].map(label_map)
map_dict = gs.set_index(['Day_of_week','hour'])['cluster'].to_dict()
data['Relation_Hour_Weekday'] = data.apply(
    lambda r: map_dict.get((r['Day_of_week'], r['hour'])), axis=1
)

# === 11) SHIFT INDICATORS ===
data['shift_1'] = data['hour'].between(13,20).astype(int)
data['shift_2'] = ((data['hour']>=21)|(data['hour']<5)).astype(int)

# === 12) TEMPERATURE FEATURES ===
# Align to short dataset
end_ts = pd.to_datetime('2024-12-24 23:00')

# Slice & then drop rows where Load is missing (but keep the last hour)
data_short = data.loc[:end_ts].copy().dropna()
data_short['temperature']       = df_temperature.reindex(data_short.index)
data_short['temperature_lag48'] = data_short['temperature'].shift(48)
data_short['temperature_lag54'] = data_short['temperature'].shift(54)

# === 13) TRAIN/VAL/TEST SPLIT ===
# val_start  = pd.to_datetime('2024-08-07')
# test_start = pd.to_datetime('2024-10-07')
# train = data_short.loc[:val_start - pd.Timedelta(days=1)].copy()
# val   = data_short.loc[val_start:test_start - pd.Timedelta(hoours=1)].copy()
# test  = data_short.loc[test_start:].copy()
full_df = data_short.copy()

# === END ===

# === 14) EXPORT FULL FEATURES ===
# Only full_df is exported
full_df.to_csv('full_features.csv')
print("Full feature set exported to 'full_features.csv'")


# === 15) EXPORT TO EXCEL ===
# Write full features into an Excel file for easy reuse
excel_path = 'full_features.xlsx'
# Directly export without ExcelWriter for simplicity
full_df.to_excel(excel_path, index=True)
print(f"Full feature set also exported to '{excel_path}'")



,Load,hour,Day_of_week,Day_of_year,month,year,week_of_year,season,is_weekend,hour_sin,...,dow_5,dow_6,Exogenous_3_off,Exogenous_4_standby,Relation_Hour_Weekday,shift_1,shift_2,temperature,temperature_lag48,temperature_lag54
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-15 00:00:00,1372.0,0,6,15,1,2023,2,2,1,0.000000,...,0,1,0,0,2,0,1,5.42,NaN,NaN
2023-01-15 01:00:00,1520.0,1,6,15,1,2023,2,2,1,0.258819,...,0,1,0,0,2,0,1,4.78,NaN,NaN
2023-01-15 02:00:00,1472.0,2,6,15,1,2023,2,2,1,0.500000,...,0,1,0,0,2,0,1,4.09,NaN,NaN
2023-01-15 03:00:00,1408.0,3,6,15,1,2023,2,2,1,0.707107,...,0,1,0,0,2,0,1,3.64,NaN,NaN
2023-01-15 04:00:00,1502.0,4,6,15,1,2023,2,2,1,0.866025,...,0,1,0,0,2,0,1,3.55,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23 20:00:00,1552.0,20,0,358,12,2024,52,2,0,-0.866025,...,0,0,0,0,2,1,0,3.50,9.02,8.01
2024-12-23 21:00:00,1274.0,21,0,358,12,2024,52,2,0,-0.707107,...,0,0,0,0,1,0,1,2.90,7.63,8.21
2024-12-23 22:00:00,703.0,22,0,358,12,2024,52,2,0,-0.500000,...,0,0,0,0,2,0,1,3.34,7.29,8.17


In [13]:
# === 14) EXPORT FULL FEATURES ===
# Only full_df is exported
full_df.to_csv('full_features.csv')
print("Full feature set exported to 'full_features.csv'")


Full feature set exported to 'full_features.csv'


In [17]:
# === 15) EXPORT TO EXCEL ===
# Write full features into an Excel file for easy reuse
excel_path = 'full_features.xlsx'
# Directly export without ExcelWriter for simplicity
full_df.to_excel(excel_path, index=True)
print(f"Full feature set also exported to '{excel_path}'")

Full feature set also exported to 'full_features.xlsx'
